In [2]:
#!pip install sharepy
import sharepy 
import os
#System variables
MAIL_NUB = os.environ.get('MAIL_NUB')
PASS_MAIL_NUB = os.environ.get('PASS_MAIL_NUB')
s = sharepy.connect("nubsieteocho.sharepoint.com", MAIL_NUB, PASS_MAIL_NUB)

In [3]:
import smtplib
import pandas as pd
from datetime import datetime, timedelta
from pandas import Timestamp

from email import encoders
from os.path import basename
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase

In [10]:
#import data with the users mails
p = s.getfile("https://nubsieteocho.sharepoint.com/sites/ODAOPS/Shared%20Documents/General/CAB%20Trackers/Coverage%20Advisor%20Tracker%20OFICIAL.xlsx"\
    ,filename = 'Coverage Advisor Tracker OFICIAL.xlsx')
ca = pd.read_excel ('Coverage Advisor Tracker OFICIAL.xlsx')

C:\Users\juanr3\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Account,Request ID,SL1,SL2,SL3,SL4,Request Type,Join Call,Sav ID,Upcoming Renewal,...,Follow Up 1,Follow Up 2,Follow Up 3,Closed Date,Method,DID,Link PAR IB Rep,Cycletime,Type,Campaign
0,TECO ENERGY,103120.0,Americas,US COMMERCIAL,COMMERCIAL EAST AREA,SOUTH EAST COMMERCIAL OPERATION,Proactive,No,203787042.0,FY22Q1,...,2021-12-02 00:00:00,2022-02-28 00:00:00,NaT,NaT,Webex,55219500,PC - TECO ENERGY - 20211014.xlsb (sharepoint.com),4.0,HS,Coverage Program - SNTC Proactive Attach
1,BLUE CROSS BLUE SHIELD OF TENNESSEE INC,102639.0,Americas,US COMMERCIAL,COMMERCIAL CENTRAL AREA,CENTRAL EAST COMMERCIAL OPERATION,Proactive,No,203729830.0,FY22Q2,...,2021-12-02 00:00:00,2022-02-28 00:00:00,NaT,2022-02-07,Webex,52708881,PC - BLUECROSS BLUESHIELD OF TENNESSEE INC - 2...,6.0,HT,ReActive
2,Summa Healthcare,102732.0,Americas,US COMMERCIAL,COMMERCIAL CENTRAL AREA,CENTRAL EAST COMMERCIAL OPERATION,Proactive,No,203857804.0,FY22Q4,...,2021-12-02 00:00:00,2022-02-21 00:00:00,NaT,NaT,Webex,44714987,PC - SUMMA HEALTHCARE - 20211016.xlsb (sharepo...,8.0,HT,Reactive
3,Infonavit,103048.0,Americas,LATIN AMERICA,MEXICO-,MEXICO PUBLIC SECTOR,Proactive,No,219015526.0,FY22Q2,...,2021-12-02 00:00:00,2022-03-07 00:00:00,NaT,NaT,Webex,55218776,PC - INSTITUTO DEL FONDO NACIONAL DE LA VIVIEN...,8.0,HS,Coverage Program - SNTC TAC Leads
4,Gobierno del Estado de Mexico,103106.0,Americas,LATIN AMERICA,MEXICO-,MEXICO PUBLIC SECTOR,Proactive,No,203722008.0,FY24Q1,...,2021-12-02 00:00:00,2022-03-07 00:00:00,NaT,NaT,Webex,55219343,PC - GOBIERNO DEL ESTADO DE MEXICO - 20211016....,8.0,HS,Coverage Program - SNTC Proactive Attach
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,PANDUIT,104341.0,Americas,US COMMERCIAL,NaN,NaN,Refresh,No,203796395.0,FY24Q2,...,NaN,NaN,NaT,NaT,Webex,NaN,https://cisco.sharepoint.com/sites/AmericasTSS...,9.0,HT,Cover the Uncovered - Coverage at Renewal
1045,"BANCO DE LA CIUDAD DE BUENOS AIRES, FUNDACION ...",112872.0,NaN,NaN,NaN,NaN,Proactive,Yes - Compass Advisor,NaN,NaN,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,-44615.0,NaN,NaN
1046,ADP LLC,111997.0,NaN,GLOBAL ENTERPRISE SEGMENT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,4.0,NaN,NaN
1047,IBM US,108728.0,NaN,GLOBAL ENTERPRISE SEGMENT,NaN,NaN,Reactive,No,203799420.0,NaN,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,1.0,HT,Reactive


In [ ]:
#for i, j in ca['PAR Received'].iteritems():
#    print(str(j) + '/// '+str(type(j)))

In [5]:


def format_date_column(serie):
    format_date = []
    for index, value in serie.iteritems():
        if type(value) != datetime and type(value) != Timestamp:  
            if type(value) == float or value != value:
                value = datetime.strptime('2030-2-2 00:00:00', '%Y-%m-%d %H:%M:%S')
                format_date.append(value)
            else: 
                value = datetime.strptime('2050-2-2 00:00:00', '%Y-%m-%d %H:%M:%S')
                format_date.append(value)
        else:
            value = datetime.strptime(str(value), '%Y-%m-%d %H:%M:%S')
            format_date.append(value)
    
    return format_date
    


### Celdas vacías

In [54]:
#Recorrido de celdas buscando valores vacios en columnas G H I J  AF/AG/AH AM AN AO
Empty_Errors  = [
    (ca['Request Type'].isnull == True) | (ca['Join Call'].isnull == True)
    | (ca['Sav ID'].isnull == True) | (ca['Upcoming Renewal'].isnull == True) | (ca['CA Name'].isnull == True) | (ca['Expired Opportunity'].isnull == True)
    | (ca['LDoS Opportunity'].isnull == True) | (ca['Next steps'].isnull == True) | (ca['DID'].isnull == True) | (ca['Link PAR IB Rep'].isnull == True)]

Empty_Errors

[False]

### Compare column R-W

In [ ]:
equality_relation = ca[['Request ID','PAR Requested','Date First Contact','PAR Received','Date Analysis','Date Invite Sent','Date Scheduled']]

In [ ]:
#for i, j in equality_relation['Date Analysis'].iteritems():
#    print(str(j) + '/// '+str(type(j)))

In [ ]:
equality_relation['PAR Requested'] = format_date_column(equality_relation['PAR Requested'])
equality_relation['Date First Contact'] = format_date_column(equality_relation['Date First Contact'])
equality_relation['PAR Received'] = format_date_column(equality_relation['PAR Received'])
equality_relation['Date Analysis'] = format_date_column(equality_relation['Date Analysis'])
equality_relation['Date Invite Sent'] = format_date_column(equality_relation['Date Analysis'])
equality_relation['Date Scheduled'] = format_date_column(equality_relation['Date Scheduled'])

In [ ]:
equality_relation['R_less_S'] = int(0)
equality_relation['S_less_T'] = int(0)
equality_relation['T_less_U'] = int(0)
equality_relation['U_less_V'] = int(0)
equality_relation['V_less_W'] = int(0)

In [ ]:
for i, value in equality_relation.iterrows():
    if value['PAR Requested'] > value['Date First Contact']:
        if value['PAR Requested'] == Timestamp('2030-2-2 00:00:00') or value['PAR Requested'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation['R_less_S'].iloc[i] = 0
        else:
            equality_relation['R_less_S'].iloc[i] = 1

    if value['Date First Contact'] > value['PAR Received']:
        if value['Date First Contact'] == Timestamp('2030-2-2 00:00:00') or value['Date First Contact'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation['S_less_T'].iloc[i] = 0
        else:
            equality_relation['S_less_T'].iloc[i] = 1  

    if value['PAR Received'] > value['Date Analysis']:
        if value['PAR Received'] == Timestamp('2030-2-2 00:00:00') or value['PAR Received'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation['T_less_U'].iloc[i] = 0  
        else:
            equality_relation['T_less_U'].iloc[i] = 1

    if value['Date Analysis'] > value['Date Invite Sent']:
        if value['Date Analysis'] == Timestamp('2030-2-2 00:00:00') or value['Date Analysis'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation['U_less_V'].iloc[i] = 0
        else:
            equality_relation['U_less_V'].iloc[i] = 1        

    if value['Date Invite Sent'] > value['Date Scheduled']:
        if value['Date Invite Sent'] == Timestamp('2030-2-2 00:00:00') or value['Date Invite Sent'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation['V_less_W'].iloc[i] = 0
        else: 
            equality_relation['V_less_W'].iloc[i] = 1 

In [ ]:
for i, val in equality_relation.iterrows():

    if val['PAR Requested'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation['PAR Requested'].iloc[i] = 'Format Error'
    else:
        equality_relation['PAR Requested'].iloc[i] = ' '

    if val['Date First Contact'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation['Date First Contact'].iloc[i] = 'Format Error'
    else:
        equality_relation['Date First Contact'].iloc[i] = ' '
    
    if val['PAR Received'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation['PAR Received'].iloc[i] = 'Format Error'
    else:
        equality_relation['PAR Received'].iloc[i] = ' '

    if val['Date Analysis'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation['Date Analysis'].iloc[i] = 'Format Error'
    else:
        equality_relation['Date Analysis'].iloc[i] = ' '
    
    if val['Date Invite Sent'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation['Date Invite Sent'].iloc[i] = 'Format Error'
    else:
        equality_relation['Date Invite Sent'].iloc[i] = ' '
    
    if val['Date Scheduled'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation['Date Scheduled'].iloc[i] = 'Format Error'
    else:
        equality_relation['Date Scheduled'].iloc[i] = ' '


### Familiar Relation

In [ ]:

familiar_relation = ca[['Request ID','Account Stage','Status','Reason Declined','Pre-Readout Effort (min)','Readout Duration (min)','Follow Up Effort (min)']]

In [ ]:
familiar_relation['Error relation X,Y columns'] = ' '
familiar_relation['Error Reason Declined'] = ' '
familiar_relation['Pre-ReadOut Empty'] = ' '
familiar_relation['Empty values in AB and AC'] = ' '



In [ ]:
for index, status in familiar_relation.iterrows():
    if type(status['Account Stage']) != float and type(status['Status']) != float:
        if status['Status'][0] != status['Account Stage'][0]:
            familiar_relation['Error relation X,Y columns'].iloc[index] = 'Error'
        else:
            if status['Account Stage'][0] == 'E' and status['Status'][2]== 'D' and type(status['Reason Declined']) == float:
                familiar_relation['Error Reason Declined'].iloc[index] = 'Error'
        
    if type(status['Pre-Readout Effort (min)']) != int:
        if type(status['Pre-Readout Effort (min)']) == float:
            if type(status['Status']) != float:
                if status['Status'][2] != 'U':
                    familiar_relation['Pre-ReadOut Empty'].iloc[index] = 'Error'

    if type(status['Account Stage']) != float:
        if status['Account Stage'][0] == 'F' and (type(status['Readout Duration (min)']) == float or type(status['Follow Up Effort (min)']) == float):
            familiar_relation['Empty values in AB and AC'].iloc[index] = 'Error'


    

In [ ]:
familiar_relation = familiar_relation.drop('Account Stage', axis=1)
familiar_relation = familiar_relation.drop('Status', axis=1)
familiar_relation = familiar_relation.drop('Reason Declined', axis=1)
familiar_relation = familiar_relation.drop('Pre-Readout Effort (min)', axis=1)
familiar_relation = familiar_relation.drop('Readout Duration (min)', axis=1)
familiar_relation = familiar_relation.drop('Follow Up Effort (min)', axis=1)

In [ ]:

familiar_relation2 = ca[["Follow Up 1","Follow Up 2","Follow Up 3","Closed Date",]]

In [ ]:
familiar_relation2['Follow Up 1'] = ' '
familiar_relation2['Follow Up 2'] = ' '
familiar_relation2['Follow Up 3'] = ' '
familiar_relation2['Closed Date'] = ' '


In [ ]:
for index, followUp1 in familiar_relation2.iterrows():
    if type(followUp1['Follow Up 1']) != float and type(followUp1['Follow Up 2']) != float:
        if followUp1['Follow Up 1'][0] != followUp1['Follow Up 2'][0]:
            familiar_relation2['Error relation X,Y columns'].iloc[index] = 'Error'
        else:
            if followUp1['Account Stage'][0] == 'E' and followUp1['Status'][2]== 'D' and type(followUp1['Reason Declined']) == float:
                followUp1['Error Reason Declined'].iloc[index] = 'Error'
        
    if type(followUp1['Pre-Readout Effort (min)']) != int:
        if type(followUp1['Pre-Readout Effort (min)']) == float:
            if type(followUp1['Status']) != float:
                if followUp1['Status'][2] != 'U':
                    familiar_relation2['Pre-ReadOut Empty'].iloc[index] = 'Error'

    if type(followUp1['Account Stage']) != float:
        if followUp1['Account Stage'][0] == 'F' and (type(followUp1['Readout Duration (min)']) == float or type(followUp1['Follow Up Effort (min)']) == float):
            familiar_relation2['Empty values in AB and AC'].iloc[index] = 'Error'


    

In [56]:
familiar_relation2 = familiar_relation2.drop('Follow Up 1', axis=1)
familiar_relation2 = familiar_relation2.drop('Follow Up 2', axis=1)
familiar_relation2 = familiar_relation2.drop('Follow Up 3', axis=1)
familiar_relation2 = familiar_relation2.drop('Closed Date ', axis=1)

NameError: name 'familiar_relation2' is not defined

### Compare column AI-AL


In [58]:
equality_relation2 = ca[['Follow Up 1','Follow Up 2','Follow Up 3','Closed Date']]

In [ ]:
#for i, j in equality_relation['Date Analysis'].iteritems():
#    print(str(j) + '/// '+str(type(j)))

In [ ]:
equality_relation2['Follow Up 1'] = format_date_column(equality_relation['Follow Up 1'])
equality_relation2['Follow Up 2'] = format_date_column(equality_relation['Follow Up 2'])
equality_relation2['Follow Up 3'] = format_date_column(equality_relation['Follow Up 3'])
equality_relation2['Closed Date'] = format_date_column(equality_relation['Closed Date'])

In [ ]:
equality_relation2['AI_less_AJ'] = int(0)
equality_relation2['AJ_less_AK'] = int(0)
equality_relation2['AK_less_AL'] = int(0)

In [ ]:
for i, value in equality_relation2.iterrows():
    if value['Follow Up 1'] > value['Follow Up 2']:
        if value['Follow Up 1'] == Timestamp('2030-2-2 00:00:00') or value['Follow Up 1'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation2['AI_less_AJ'].iloc[i] = 0
        else:
            equality_relation['AI_less_AJ'].iloc[i] = 1

    if value['Follow Up 2'] > value['Follow Up 3']:
        if value['Follow Up 2'] == Timestamp('2030-2-2 00:00:00') or value['Follow Up 2'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation['AJ_less_AK'].iloc[i] = 0
        else:
            equality_relation['AJ_less_AK'].iloc[i] = 1  

    if value['Follow Up 3'] > value['Closed Date']:
        if value['Follow Up 3'] == Timestamp('2030-2-2 00:00:00') or value['Follow Up 3'] == Timestamp('2050-2-2 00:00:00'):
            equality_relation['AK_less_AL'].iloc[i] = 0  
        else:
            equality_relation['AK_less_AL'].iloc[i] = 1

In [ ]:
for i, val in equality_relation.iterrows():

    if val['Follow Up 1'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation['Follow Up 1'].iloc[i] = 'Format Error'
    else:
        equality_relation['Follow Up 1'].iloc[i] = ' '

    if val['Follow Up 2'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation['Follow Up 2'].iloc[i] = 'Format Error'
    else:
        equality_relation['Follow Up 2'].iloc[i] = ' '
    
    if val['Follow Up 3'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation['Follow Up 3'].iloc[i] = 'Format Error'
    else:
        equality_relation['Follow Up 3'].iloc[i] = ' '

    if val['Closed Date'] == Timestamp('2050-2-2 00:00:00'):
        equality_relation['Closed Date'].iloc[i] = 'Format Error'
    else:
        equality_relation['Closed Date'].iloc[i] = ' '
    